# Toy problem for debugging Dual Decomposition

$$
\begin{align*}
\max_{x_k,y} \quad & \sum_k f_k'y + \sum_k c_k'x_k \\
\mathrm{s.t.} \quad & A_k x_k + B_k y \leq d_k, \\
& y\in \{0,1\}.
\end{align*}
$$

$$
\begin{align*}
\max_{x_k,y_k} \quad & \sum_k f_k'y + \sum_k c_k'x_k \\
\mathrm{s.t.} \quad & A_k x_k + B_k y_k \leq d_k, \\
& \sum_k H_k y_k = 0, \\
& y_k\in \{0,1\}.
\end{align*}
$$

In [1]:
RELAX_LP = True

In [2]:
from __future__ import division
from cobra import Model
from dynamicme.optimize import Variable, Constraint
from six import iteritems
import numpy as np

### Example from
http://www.optirisk-systems.com/manuals/FortspManual.pdf

$$
\begin{align*}
\max_{x,y} \quad & \sum_k P_k \left( \sum_c p^s_c x^s_{c,k} - p^b_c x^b_{c,k}  - \sum_c c_c a_c \right)\\
\mathrm{s.t.} \quad & \sum_c a_c \leq A \\
& Y_{c,k}  a_c - x^s_{c,k} + x^b_{c,k} \geq R_c \\
& x^s_{\mathrm{beet},k} \leq Q(\mathrm{beet}) \\
& x^s_{\mathrm{beet},c} + s^x_k \leq Y_{\mathrm{beet},k} a_\mathrm{beet}
\end{align*}
$$

In [3]:
from dynamicme.decomposition import LagrangeMaster, LagrangeSubmodel

In [4]:
#------------------------------------------------------------
# Data
crops = ['wheat','corn','beet']
total_area = 500.  # Total acres
prob_dict = {'below':1./3, 'average':1./3, 'above':1./3}
# Yields (tons/acre) over scenarios
yield_dict = {}
yield_dict['below'] = {'wheat':2., 'corn':2.4, 'beet':16.}
yield_dict['average'] = {'wheat':2.5, 'corn':3., 'beet':20.}
yield_dict['above'] = {'wheat':3., 'corn':3.6, 'beet':24.}
# Planting costs
plantcost_dict={'wheat':150, 'corn':230, 'beet':260}   # $/ton
sellprice_dict = {'wheat':170, 'corn':150, 'beet':36}  # $/ton
excess_sell_price = 10    # beet sell price ($/ton) when quota exceeded
beet_quota = 6000   # tons
buyprice_dict = {'wheat':238, 'corn':210, 'beet':100}  # $/ton
req_dict = {'wheat':200., 'corn':240., 'beet':0.} # tons required to feed cattle
#------------------------------------------------------------
# Area is complicating (first-stage) variable

In [5]:
# Create subproblems
UB = 1e4
sub_dict = {}
for scen,yields in iteritems(yield_dict):
    # Each subproblem gets its own copy of first-stage variables
    area_dict = {c:Variable('area_%s'%c, lower_bound=0., upper_bound=1e6,
        objective_coefficient=prob_dict[scen]*plantcost_dict[c]) for c in crops}
    for y in area_dict.values():
        y.variable_kind = 'integer'
    mdl = Model('sub')
    mdl.add_reactions(area_dict.values())
    # Global constraint: sum_a <= Area
    cons_area = Constraint('total_area')
    cons_area._bound = total_area
    cons_area._constraint_sense = 'L'
    mdl.add_metabolites(cons_area)
    for y in area_dict.values():
        y.add_metabolites({cons_area:1.}, combine=False)

    # MAX sell price
    x_excess = Variable('sell_excess_beet', lower_bound=0., upper_bound=UB)
    x_excess.objective_coefficient = -prob_dict[scen]*excess_sell_price    
    mdl.add_reaction(x_excess)
    
    for c in crops:
        ### Scenario variables x: tons sold, bought, excess sold per scenario
        x_sell = Variable('sell_%s'%(c), lower_bound=0., upper_bound=UB)
        x_buy  =  Variable('buy_%s'%(c), lower_bound=0., upper_bound=UB)
        mdl.add_reactions([x_sell,x_buy])        
        x_sell.objective_coefficient = -prob_dict[scen]*sellprice_dict[c]
        x_buy.objective_coefficient = prob_dict[scen]*buyprice_dict[c]
        
        ### Scenario constraint: Y_ck*ac - xs_ck + xb_ck >= Req_c        
        cons = Constraint('required_%s'%c)
        cons._bound = req_dict[c]
        cons._constraint_sense = 'G'        
        mdl.add_metabolites(cons)
        area = area_dict[c]
        area.add_metabolites({cons:yield_dict[scen][c]})
        x_sell.add_metabolites({cons:-1.})
        x_buy.add_metabolites({cons:1.})
        
        ### Beet quota
        if c=='beet':
            x_sell.upper_bound = beet_quota
            # xs_beet,k + xexcess_k <= Yield_beet,k * area_beet
            cons_beet = Constraint('beet_balance')
            cons_beet._bound = 0.
            cons_beet._constraint_sense = 'L'
            x_sell.add_metabolites({cons_beet:1.})
            x_excess.add_metabolites({cons_beet:1.})
            area_dict[c].add_metabolites({cons_beet:-yield_dict[scen][c]})
    
    sub = LagrangeSubmodel(mdl, scen)
    sub_dict[scen] = sub

In [6]:
master = LagrangeMaster(mdl)
master._INF = 1e7
master.add_submodels(sub_dict)
master._z.LB = -master._INF
master._z.UB = master._INF
master.model.update()

In [7]:
obj = master.model.getObjective()
obj

<gurobi.QuadExpr: z + [ -0.5 u_00 ^ 2 + -0.5 u_01 ^ 2 + -0.5 u_02 ^ 2 + -0.5 u_10 ^ 2 + -0.5 u_11 ^ 2 + -0.5 u_12 ^ 2 ]>

In [8]:
for v in master.model.getVars():
    print(v.VarName, v.LB, v.UB)

('z', -10000000.0, 10000000.0)
('tk_below', -10000000.0, 10000000.0)
('tk_average', -10000000.0, 10000000.0)
('tk_above', -10000000.0, 10000000.0)
('u_00', -10000000.0, 10000000.0)
('u_01', -10000000.0, 10000000.0)
('u_02', -10000000.0, 10000000.0)
('u_10', -10000000.0, 10000000.0)
('u_11', -10000000.0, 10000000.0)
('u_12', -10000000.0, 10000000.0)


In [9]:
for sub_ind,sub in iteritems(sub_dict):
    print('H_%s'%sub_ind)
    print(sub._H.todense())

H_below
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
H_average
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
H_above
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]


In [10]:
cc = master.model.getConstrs()[0]
print(master.model.getRow(cc), cc.Sense, cc.RHS)

(<gurobi.LinExpr: z + -1.0 tk_below + -1.0 tk_average + -1.0 tk_above>, '<', 0.0)


In [11]:
master.model.getObjective()

<gurobi.QuadExpr: z + [ -0.5 u_00 ^ 2 + -0.5 u_01 ^ 2 + -0.5 u_02 ^ 2 + -0.5 u_10 ^ 2 + -0.5 u_11 ^ 2 + -0.5 u_12 ^ 2 ]>

In [12]:
cz = master.model.getConstrByName('z_cut')
print(master.model.getRow(cz), cz.Sense, cz.RHS)

(<gurobi.LinExpr: z + -1.0 tk_below + -1.0 tk_average + -1.0 tk_above>, '<', 0.0)


In [13]:
from dynamicme.callback_gurobi import cb_benders_multi

In [14]:
# Need to scale variables? Since values are huge?

In [15]:
master.model.Params.Presolve = 0
# master.model.Params.ScaleFlag = 2
# master.model.Params.NumericFocus = 3
master.model.Params.BarHomogeneous = 1

# master.model.Params.Method = 0  # primal simplex
# master.model.Params.Method = 1  # dual simplex
master.model.Params.Method = 2  # barrier

master.model.Params.OutputFlag=0
master.max_iter = 1
master.print_iter=1
master.verbosity = 2

master.delta_mult = 0.5   #0.1
master.delta_min  = 1e-20 #0

if RELAX_LP:
    uopt = master.solve_relaxed()
else:    
    uopt = master.optimize() 

        Iter     Dual UB     Feas UB     Best LB         gap   relgap(%)       delta     time(s)
           0       1e+07      1e+100  -1.154e+05   1.012e+07       101.2           1    0.006578


In [16]:
master._us

[<gurobi.Var u_00>,
 <gurobi.Var u_01>,
 <gurobi.Var u_02>,
 <gurobi.Var u_10>,
 <gurobi.Var u_11>,
 <gurobi.Var u_12>]

In [17]:
sub._H.shape

(6, 3)

In [18]:
for sub_ind,sub in iteritems(sub_dict):
    print('%s'%sub_ind)    
    print([(x.VarName, x.Obj, x.X) for x in sub._ys])
    print('-'*40)

below
[('area_beet', 86.66666666666666, 375.0), ('area_corn', 76.66666666666666, 25.0), ('area_wheat', 50.0, 100.0)]
----------------------------------------
average
[('area_beet', 86.66666666666666, 300.0), ('area_corn', 76.66666666666666, 80.0), ('area_wheat', 50.0, 120.0)]
----------------------------------------
above
[('area_beet', 86.66666666666666, 250.0), ('area_corn', 76.66666666666666, 66.66666666666667), ('area_wheat', 50.0, 183.33333333333331)]
----------------------------------------


In [19]:
if RELAX_LP:
    uopt = master.solve_relaxed()
else:    
    uopt = master.optimize() 

        Iter     Dual UB     Feas UB     Best LB         gap   relgap(%)       delta     time(s)
           0  -4.869e+04      1e+100  -3.337e+05    2.85e+05       585.2           1    0.004544


In [20]:
for sub_ind,sub in iteritems(sub_dict):
    print('%s'%sub_ind)    
    print([(x.VarName, x.Obj, x.X) for x in sub._ys])
    print('-'*40)

below
[('area_beet', 486.66666666666663, -0.0), ('area_corn', -116.66666666728756, 100.0), ('area_wheat', -156.66666666790843, 400.0)]
----------------------------------------
average
[('area_beet', -63.33333333333334, 500.0), ('area_corn', 186.66666666666666, -0.0), ('area_wheat', 90.0, -0.0)]
----------------------------------------
above
[('area_beet', -163.33333333333334, 500.0), ('area_corn', 160.00000000062087, -0.0), ('area_wheat', 216.66666666790843, -0.0)]
----------------------------------------


In [21]:
uz = master.model.getConstrs()[1]
print(master.model.getRow(uz), uz.Sense, uz.RHS)

(<gurobi.LinExpr: u_00 + u_01 + u_02 + u_10 + u_11 + u_12>, '=', 0.0)


In [22]:
master.model.Params.FeasibilityTol=1e-9
master.max_iter=100
master.verbosity=0
if RELAX_LP:
    uopt = master.solve_relaxed()
else:    
    uopt = master.optimize() 

        Iter     Dual UB     Feas UB     Best LB         gap   relgap(%)       delta     time(s)
           0  -9.971e+04      1e+100   -1.89e+05   8.932e+04       89.57           1    0.005458
           1  -1.067e+05      1e+100  -1.138e+05        7138        6.69         0.5    0.010962
           2  -1.077e+05      1e+100  -1.119e+05        4240       3.937        0.25    0.016816
           3  -1.084e+05      1e+100  -1.084e+05   1.669e-06    1.54e-09       0.125    0.022522
relgap (1.54013e-11) <= gaptol (0.0001). Finished.
sum_k Hk*yk: [ 300.           20.         -320.           50.          -83.33333333
   33.33333333]


In [23]:
master.uopt.sum()

1.862645149230957e-09

In [24]:
for sub_ind,sub in iteritems(sub_dict):
    print('%s'%sub_ind)    
    print([(x.VarName, x.Obj, x.X) for x in sub._ys])
    print('-'*40)

below
[('area_beet', 105.33333334388831, 300.0), ('area_corn', 81.33333331408599, 100.0), ('area_wheat', 26.666666677221656, 100.0)]
----------------------------------------
average
[('area_beet', 131.55555554355183, 0.0), ('area_corn', 48.55555557894209, 80.0), ('area_wheat', 33.22222221083939, 420.0)]
----------------------------------------
above
[('area_beet', 23.111111112559826, 250.0), ('area_corn', 100.11111110697189, 183.33333333333334), ('area_wheat', 90.11111111193895, 66.66666666666667)]
----------------------------------------


In [25]:
cons = master.model.getConstrs()[1]
print(master.model.getRow(cons),cons.Sense,cons.RHS)
cons = master.model.getConstrs()[2]
print(master.model.getRow(cons),cons.Sense,cons.RHS)
cons = master.model.getConstrs()[3]
print(master.model.getRow(cons),cons.Sense,cons.RHS)

(<gurobi.LinExpr: u_00 + u_01 + u_02 + u_10 + u_11 + u_12>, '=', 0.0)
(<gurobi.LinExpr: tk_below + -375.0 u_00 + -25.0 u_01 + -100.0 u_02 + -375.0 u_10 + -25.0 u_11 + -100.0 u_12>, '<', -19983.333333333336)
(<gurobi.LinExpr: tk_average + 300.0 u_00 + 80.0 u_01 + 120.0 u_02>, '<', -39533.33333333334)


In [26]:
ncons=len(master.model.getConstrs())
cons = master.model.getConstrs()[ncons-3]
print(master.model.getRow(cons),cons.Sense,cons.RHS)
cons = master.model.getConstrs()[ncons-2]
print(master.model.getRow(cons),cons.Sense,cons.RHS)
cons = master.model.getConstrs()[ncons-1]
print(master.model.getRow(cons),cons.Sense,cons.RHS)

(<gurobi.LinExpr: tk_below + -300.0 u_00 + -100.0 u_01 + -100.0 u_02 + -300.0 u_10 + -100.0 u_11 + -100.0 u_12>, '<', -18933.333333333336)
(<gurobi.LinExpr: tk_average + 80.0 u_01 + 420.0 u_02>, '<', -21033.333333333332)
(<gurobi.LinExpr: tk_above + 250.0 u_10 + 183.333333333 u_11 + 66.6666666667 u_12>, '<', -53944.444444444445)


In [27]:
l = [[1,2,3],[1,2,3]]
[ii for i in [[1,2,3],[1,2,3]] for ii in i]

[1, 2, 3, 1, 2, 3]

In [28]:
cons = master.model.getConstrs()[1]
print(master.model.getRow(cons),cons.Sense,cons.RHS)
cons = master.model.getConstrs()[2]
print(master.model.getRow(cons),cons.Sense,cons.RHS)
cons = master.model.getConstrs()[3]
print(master.model.getRow(cons),cons.Sense,cons.RHS)

(<gurobi.LinExpr: u_00 + u_01 + u_02 + u_10 + u_11 + u_12>, '=', 0.0)
(<gurobi.LinExpr: tk_below + -375.0 u_00 + -25.0 u_01 + -100.0 u_02 + -375.0 u_10 + -25.0 u_11 + -100.0 u_12>, '<', -19983.333333333336)
(<gurobi.LinExpr: tk_average + 300.0 u_00 + 80.0 u_01 + 120.0 u_02>, '<', -39533.33333333334)


In [29]:
print(master._z.X)
master.model.getObjective()

-108389.999999


<gurobi.QuadExpr: z + [ -0.0625 u_00 ^ 2 + -0.0625 u_01 ^ 2 + -0.0625 u_02 ^ 2 + -0.0625 u_10 ^ 2 + -0.0625 u_11 ^ 2 + -0.0625 u_12 ^ 2 ]>

In [30]:
master.model

<gurobi.Model Continuous instance master: 17 constrs, 10 vars, Parameter changes: FeasibilityTol=1e-09, Method=2, BarHomogeneous=1, LogFile=gurobi.log, Presolve=0, OutputFlag=0>

In [31]:
cons = master.model.getConstrs()[1]
print(master.model.getRow(cons), cons.Sense, cons.RHS)
cons = master.model.getConstrs()[2]
print(master.model.getRow(cons), cons.Sense, cons.RHS)
cons = master.model.getConstrs()[3]
print(master.model.getRow(cons), cons.Sense, cons.RHS)

(<gurobi.LinExpr: u_00 + u_01 + u_02 + u_10 + u_11 + u_12>, '=', 0.0)
(<gurobi.LinExpr: tk_below + -375.0 u_00 + -25.0 u_01 + -100.0 u_02 + -375.0 u_10 + -25.0 u_11 + -100.0 u_12>, '<', -19983.333333333336)
(<gurobi.LinExpr: tk_average + 300.0 u_00 + 80.0 u_01 + 120.0 u_02>, '<', -39533.33333333334)


In [32]:
cons = master.model.getConstrs()[4]
print(master.model.getRow(cons), cons.Sense, cons.RHS)
cons = master.model.getConstrs()[5]
print(master.model.getRow(cons), cons.Sense, cons.RHS)
cons = master.model.getConstrs()[6]
print(master.model.getRow(cons), cons.Sense, cons.RHS)

(<gurobi.LinExpr: tk_above + 250.0 u_10 + 66.6666666667 u_11 + 183.333333333 u_12>, '<', -55888.88888888889)
(<gurobi.LinExpr: tk_below + -100.0 u_01 + -400.0 u_02 + -100.0 u_11 + -400.0 u_12>, '<', -6333.333333333336)
(<gurobi.LinExpr: tk_average + 500.0 u_00>, '<', -9333.333333333336)


In [33]:
ncons = len(master.model.getConstrs())
cons = master.model.getConstrs()[ncons-3]
print(master.model.getRow(cons), cons.Sense, cons.RHS)
cons = master.model.getConstrs()[ncons-2]
print(master.model.getRow(cons), cons.Sense, cons.RHS)
cons = master.model.getConstrs()[ncons-1]
print(master.model.getRow(cons), cons.Sense, cons.RHS)

(<gurobi.LinExpr: tk_below + -300.0 u_00 + -100.0 u_01 + -100.0 u_02 + -300.0 u_10 + -100.0 u_11 + -100.0 u_12>, '<', -18933.333333333336)
(<gurobi.LinExpr: tk_average + 80.0 u_01 + 420.0 u_02>, '<', -21033.333333333332)
(<gurobi.LinExpr: tk_above + 250.0 u_10 + 183.333333333 u_11 + 66.6666666667 u_12>, '<', -53944.444444444445)


In [34]:
for sub_ind,sub in iteritems(sub_dict):
    print('%s'%sub_ind)
    print(sub.model.getObjective())
    print(sub._ys)
    print('-'*40)

below
<gurobi.LinExpr: 105.333333344 area_beet + 81.3333333141 area_corn + 26.6666666772 area_wheat + -3.33333333333 sell_excess_beet + -56.6666666667 sell_wheat + 79.3333333333 buy_wheat + -50.0 sell_corn + 70.0 buy_corn + -12.0 sell_beet + 33.3333333333 buy_beet>
[<gurobi.Var area_beet (value 300.0)>, <gurobi.Var area_corn (value 100.0)>, <gurobi.Var area_wheat (value 100.0)>]
----------------------------------------
average
<gurobi.LinExpr: 131.555555544 area_beet + 48.5555555789 area_corn + 33.2222222108 area_wheat + -3.33333333333 sell_excess_beet + -56.6666666667 sell_wheat + 79.3333333333 buy_wheat + -50.0 sell_corn + 70.0 buy_corn + -12.0 sell_beet + 33.3333333333 buy_beet>
[<gurobi.Var area_beet (value 0.0)>, <gurobi.Var area_corn (value 80.0)>, <gurobi.Var area_wheat (value 420.0)>]
----------------------------------------
above
<gurobi.LinExpr: 23.1111111126 area_beet + 100.111111107 area_corn + 90.1111111119 area_wheat + -3.33333333333 sell_excess_beet + -56.6666666667 sell

## Check answer: should be 
- {'wheat':170, 'corn':80, 'beet':250}
- objval = (-)108390

In [35]:
master._z

<gurobi.Var z (value -108389.999999)>

In [36]:
x_dict = {}
for scen,sub in iteritems(master.sub_dict):
    #sub.update_obj(yopt)
    #sub.model.optimize()
    #x_dict[scen] = {v.VarName:v.X for v in sub.model.getVars()}
    x_dict[scen] = sub.x_dict

In [37]:
x_dict

{'above': {'area_beet': 250.0,
  'area_corn': 183.33333333333334,
  'area_wheat': 66.66666666666667,
  'buy_beet': 0.0,
  'buy_corn': 0.0,
  'buy_wheat': 0.0,
  'sell_beet': 6000.0,
  'sell_corn': 420.0,
  'sell_excess_beet': 0.0,
  'sell_wheat': 0.0},
 'average': {'area_beet': 0.0,
  'area_corn': 80.0,
  'area_wheat': 420.0,
  'buy_beet': 0.0,
  'buy_corn': 0.0,
  'buy_wheat': 0.0,
  'sell_beet': 0.0,
  'sell_corn': 0.0,
  'sell_excess_beet': 0.0,
  'sell_wheat': 850.0},
 'below': {'area_beet': 300.0,
  'area_corn': 100.0,
  'area_wheat': 100.0,
  'buy_beet': 0.0,
  'buy_corn': 0.0,
  'buy_wheat': 0.0,
  'sell_beet': 4800.0,
  'sell_corn': 0.0,
  'sell_excess_beet': 0.0,
  'sell_wheat': 0.0}}

In [38]:
Hys = []
sub_objs = []
for sub_ind,sub in iteritems(sub_dict):
    yk = np.array([sub.x_dict[x.VarName] for x in sub._ys])
    Hyk = sub._H*yk
    Hys.append(Hyk)    
    sub_objs.append(sub.model.ObjVal)
    print('Scenario: %s'%sub_ind)
    print('yk = %s'%[(k,v) for k,v in iteritems(sub.x_dict)])
    print('Hk*yk =%s' % Hyk)
    
print("\nsum_k Hk*yk=")
print(sum(Hys))
print("sum sum Hk*yk=%s"%sum(sum(Hys)))
print("\n u = %s" % master.uopt)

Scenario: below
yk = [('buy_wheat', 0.0), ('area_beet', 300.0), ('area_corn', 100.0), ('sell_wheat', 0.0), ('buy_corn', 0.0), ('sell_excess_beet', 0.0), ('sell_corn', 0.0), ('sell_beet', 4800.0), ('buy_beet', 0.0), ('area_wheat', 100.0)]
Hk*yk =[300. 100. 100. 300. 100. 100.]
Scenario: average
yk = [('buy_wheat', 0.0), ('area_beet', 0.0), ('area_corn', 80.0), ('sell_wheat', 850.0), ('buy_corn', 0.0), ('sell_excess_beet', 0.0), ('sell_corn', 0.0), ('sell_beet', 0.0), ('buy_beet', 0.0), ('area_wheat', 420.0)]
Hk*yk =[   0.  -80. -420.    0.    0.    0.]
Scenario: above
yk = [('buy_wheat', 0.0), ('area_beet', 250.0), ('area_corn', 183.33333333333334), ('sell_wheat', 0.0), ('buy_corn', 0.0), ('sell_excess_beet', 0.0), ('sell_corn', 420.0), ('sell_beet', 6000.0), ('buy_beet', 0.0), ('area_wheat', 66.66666666666667)]
Hk*yk =[   0.            0.            0.         -250.         -183.33333333
  -66.66666667]

sum_k Hk*yk=
[ 300.           20.         -320.           50.          -83.3333333

In [39]:
Hys = []
sub_objs = []
cxs = []
fys = []
for sub_ind,sub in iteritems(master.sub_dict):
    xk = np.array([sub.x_dict[x.VarName] for x in sub._xs])
    yk = np.array([sub.x_dict[x.VarName] for x in sub._ys])
    Hyk = sub._H*yk
    Hys.append(Hyk)    
    sub_objs.append(sub.model.ObjVal)
    cx = np.dot(sub._cx, xk)
    fy = np.dot(sub._fy, yk)
    cxs.append(cx)
    fys.append(fy)
    print('Scenario: %s'%sub_ind)
    print('c*x = %s' % cx)
    print('f*y = %s' % fy)
    print('cx + fy = %s'%(cx+fy))    
    uHy = np.dot(master.uopt, sub._H*yk)
    print("u*Hk*yk = %s" % (uHy))
    print('Sub obj val: %s' % sub.model.ObjVal)    
    print('-'*40)

# print("\nsum_k Hk*yk=")
# print(sum(Hys))
# print("sum sum Hk*yk=%s"%sum(sum(Hys)))
# print("\n u = %s" % master.uopt)
print("sum_k sub obj val: %s" % sum(sub_objs))
print("sum_k cx + fy = %s" % (sum(cxs)+sum(fys)))

Scenario: below
c*x = -57600.0
f*y = 38666.666666666664
cx + fy = -18933.333333333336
u*Hk*yk = 3733.3333356305957
Sub obj val: -15199.9999977
----------------------------------------
Scenario: average
c*x = -48166.666666666664
f*y = 27133.333333333332
cx + fy = -21033.333333333332
u*Hk*yk = -9295.555558465421
Sub obj val: -30328.8888918
----------------------------------------
Scenario: above
c*x = -93000.0
f*y = 39055.555555555555
cx + fy = -53944.444444444445
u*Hk*yk = -8916.66666700815
Sub obj val: -62861.1111115
----------------------------------------
sum_k sub obj val: -108390.000001
sum_k cx + fy = -93911.1111111111


In [40]:
for sub_ind,sub in iteritems(master.sub_dict):
    sub.update_obj(master.uopt)
    sub.optimize()
    print('-'*40)
    print('Scenario: %s'%sub_ind)
#     print('f: %s' % sub._fy)
    print('yk = %s' % {v.VarName:v.X for v in sub._ys})
#     print('xk = %s' % {v.VarName:v.X for v in sub._xs})

----------------------------------------
Scenario: below
yk = {'area_beet': 300.0, 'area_corn': 100.0, 'area_wheat': 100.0}
----------------------------------------
Scenario: average
yk = {'area_beet': 0.0, 'area_corn': 80.0, 'area_wheat': 420.0}
----------------------------------------
Scenario: above
yk = {'area_beet': 250.0, 'area_corn': 183.33333333333334, 'area_wheat': 66.66666666666667}


In [41]:
uk = master.uopt 
uk

array([-44.88888888,  28.11111109,  16.77777779,  63.55555555,
       -23.44444444, -40.11111111])

In [42]:
for sub in sub_dict.values():
    print(sub._H.todense())

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]


In [43]:
master._us

[<gurobi.Var u_00 (value -44.8888888769)>,
 <gurobi.Var u_01 (value 28.1111110877)>,
 <gurobi.Var u_02 (value 16.7777777892)>,
 <gurobi.Var u_10 (value 63.5555555541)>,
 <gurobi.Var u_11 (value -23.4444444403)>,
 <gurobi.Var u_12 (value -40.1111111119)>]

In [44]:
uk*sub._H

array([-63.55555555,  23.44444444,  40.11111111])

In [45]:
from gurobipy import LinExpr
LinExpr(uk*sub._H, sub._ys)

<gurobi.LinExpr: -63.5555555541 area_beet + 23.4444444403 area_corn + 40.1111111119 area_wheat>

## Check that all constraints satisfied

In [50]:
# Total area
for sub_ind,sub in iteritems(sub_dict):
    yopt = [v.X for v in sub._ys]
    print('Scenario=%s. Total area constraint:'%sub_ind, sum(yopt) <= total_area+master.feastol)

('Scenario=below. Total area constraint:', True)
('Scenario=average. Total area constraint:', True)
('Scenario=above. Total area constraint:', True)


In [47]:
# Cattle feed crop requirement
print("Cattle feed crop requirement met?\n%s"%('-'*40))
for scen,sub in iteritems(sub_dict):
    y_dict = {v.VarName:v.X for v in sub._ys}
    for c,req in iteritems(req_dict):
        area = y_dict['area_%s'%c]
        xsell = x_dict[scen]['sell_%s'%(c)]
        xbuy = x_dict[scen]['buy_%s'%(c)]
        lhs = yield_dict[scen][c]*area - xsell + xbuy
        rhs = req
        print('%s, %s: %g >= %g, %s'%(scen,c,lhs,rhs,lhs>=rhs))

Cattle feed crop requirement met?
----------------------------------------
below, beet: 0 >= 0, True
below, corn: 240 >= 240, True
below, wheat: 200 >= 200, True
average, beet: 0 >= 0, True
average, corn: 240 >= 240, True
average, wheat: 200 >= 200, True
above, beet: 0 >= 0, True
above, corn: 240 >= 240, True
above, wheat: 200 >= 200, True
